In [1]:
import numpy as np
import cv2
from PIL import Image, ImageEnhance
import pathlib
import random

In [2]:
path = pathlib.Path().absolute()
path = str(path)


In [3]:
cap = cv2.VideoCapture(path + '/../data/test.mp4')


In [4]:
ret, frame1 = cap.read()
rand = np.random.uniform(0.5, 1.5)
frame1 = Image.fromarray(frame1).crop((0, 160, 640, 370)).resize((150,150))
prvs = cv2.cvtColor(np.float32(frame1),cv2.COLOR_BGR2GRAY)

hsv = np.zeros_like(frame1)
hsv[...,1] = 255

In [5]:
count = 0

while(1):
    ret, frame2 = cap.read()
    if not ret:
        break
    if random.random() > 0.6:
        rand = np.random.uniform(0.5, 1.5)
        frame2 = ImageEnhance.Brightness(Image.fromarray(frame2)).enhance(rand)
        frame2 = frame2.crop((0, 160, 640, 370)).resize((150,150))
    else:
        frame2 = Image.fromarray(frame2).crop((0, 160, 640, 370)).resize((150,150))

    
    next = cv2.cvtColor(np.float32(frame2),cv2.COLOR_BGR2GRAY)

    flow = cv2.calcOpticalFlowFarneback(prvs,next, None, 0.5, 1, 15, 2, 5, 1.2, 0)

    mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])
    hsv[...,0] = ang*180/np.pi/2
    hsv[...,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
    rgb = cv2.cvtColor(hsv,cv2.COLOR_HSV2BGR)

    prvs = next
    cv2.imwrite(path + "/../data/frames/test-of/%d.jpg" % count, rgb)
    count += 1

cap.release()
